In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

from sklearn.preprocessing import StandardScaler,MinMaxScaler 


In [4]:
TIME_STEPS=500

path = "train.csv"
text_file_reader = pd.read_csv(path, float_precision='round_trip', header=None, chunksize = TIME_STEPS)

In [16]:


def get_batch_data():
    columns = ['acoustic_data','time_to_failure']
    features = ['acoustic_data']

    #print("DEBUGGGGG")
    #print(path)
    df = text_file_reader.get_chunk(TIME_STEPS)
    df.columns = columns
    #df['rolling_100'] = df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    df['acoustic_data'] = df['acoustic_data'].astype(float)
    df[features] = StandardScaler().fit_transform(df[features])
    print("debug", df['time_to_failure'].values[-1])
    return df[features].values.reshape(1,500,1), np.array(df['time_to_failure'].values[-1]).reshape(1,1)


def get_generated_batch(sequence_idx):
    return get_batch_data()

In [17]:
from keras.utils import Sequence

class MY_Generator(Sequence):

    def __init__(self, split_filenames):
        self.split_filenames = split_filenames

    #This function computes the number of batches that this generator is supposed to produce. 
    #So, we divide the number of total samples by the batch_size and return that value.    
    def __len__(self):
        return 1258290

    #Here, given the batch numberidx you need to put together a list that consists of data 
    #batch and the ground-truth (GT). In this example, we read a batch images of size 
    #self.batch and return an array of form[image_batch, GT]
    def __getitem__(self, idx):
        train,Y = get_generated_batch(idx)
        #print("idx",idx)
        #print("LOLILOL")
        print(train.shape, Y.shape)

        return (train,Y)

In [18]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


dropout=0.2


my_model = Sequential()

my_model.add(LSTM(use_bias = True,unit_forget_bias=True,\
                  units = 3,\
                  stateful=True,batch_input_shape=(1,TIME_STEPS,1),\
                  #dropout=dropout,recurrent_dropout=dropout,
                  ))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=0, verbose=0)
]

#history = my_model.fit(train, Y, batch_size=BATCH_SIZE, epochs=100, shuffle=False,
                      #validation_data=(val_data,y_val), callbacks=callbacks
                    #  )
my_training_batch_generator = MY_Generator("split")
        
my_model.fit_generator(generator=my_training_batch_generator,
                                          steps_per_epoch=1258290,
                                          epochs=10,
                                          shuffle=False,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          #use_multiprocessing=True,
                                          #workers=8,
                                          #max_queue_size=32
                      )

import matplotlib.pyplot as plt


#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (1, 3)                    60        
_________________________________________________________________
dense_5 (Dense)              (1, 1)                    4         
Total params: 64
Trainable params: 64
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
debug 1.4680995399
(1, 500, 1) (1, 1)
debug 1.4680989899
(1, 500, 1) (1, 1)
debug 1.4680984399
(1, 500, 1) (1, 1)
debug 1.4680978899
(1, 500, 1) (1, 1)
debug 1.4680973399
(1, 500, 1) (1, 1)
debug 1.4680967899
(1, 500, 1) (1, 1)
debug 1.4680962399
(1, 500, 1) (1, 1)
debug 1.4680956899
(1, 500, 1) (1, 1)
debug 1.4669996455
(1, 500, 1) (1, 1)
debug 1.4669990955
(1, 500, 1) (1, 1)
debug 1.4669985455
(1, 500, 1) (1, 1)
debug 1.4669979955
(1, 500, 1) (1, 1)
      1/1258290 [..............................] - ETA: 451:43:44 - loss: 1

     49/1258290 [..............................] - ETA: 55:31:23 - loss: 1.3998 - mean_absolute_error: 1.3998debug 1.4605980791
(1, 500, 1) (1, 1)
     50/1258290 [..............................] - ETA: 55:19:30 - loss: 1.3992 - mean_absolute_error: 1.3992debug 1.4605975291
(1, 500, 1) (1, 1)
     51/1258290 [..............................] - ETA: 55:08:56 - loss: 1.3948 - mean_absolute_error: 1.3948debug 1.4605969791
(1, 500, 1) (1, 1)
     52/1258290 [..............................] - ETA: 54:57:59 - loss: 1.3874 - mean_absolute_error: 1.3874debug 1.4605964291
(1, 500, 1) (1, 1)
     53/1258290 [..............................] - ETA: 55:03:06 - loss: 1.3808 - mean_absolute_error: 1.3808debug 1.4605958791
(1, 500, 1) (1, 1)
     54/1258290 [..............................] - ETA: 55:06:17 - loss: 1.3822 - mean_absolute_error: 1.3822debug 1.4595998347
(1, 500, 1) (1, 1)
     55/1258290 [..............................] - ETA: 55:05:05 - loss: 1.3901 - mean_absolute_error: 1.3901debug 1.4

    104/1258290 [..............................] - ETA: 52:04:51 - loss: 1.2631 - mean_absolute_error: 1.2631debug 1.4531993683
(1, 500, 1) (1, 1)
    105/1258290 [..............................] - ETA: 52:10:17 - loss: 1.2592 - mean_absolute_error: 1.2592debug 1.4531988183
(1, 500, 1) (1, 1)
    106/1258290 [..............................] - ETA: 52:12:27 - loss: 1.2547 - mean_absolute_error: 1.2547debug 1.4531982683
(1, 500, 1) (1, 1)
    107/1258290 [..............................] - ETA: 52:09:51 - loss: 1.2551 - mean_absolute_error: 1.2551debug 1.4531977183
(1, 500, 1) (1, 1)
    108/1258290 [..............................] - ETA: 52:07:52 - loss: 1.2496 - mean_absolute_error: 1.2496debug 1.4531971683
(1, 500, 1) (1, 1)
    109/1258290 [..............................] - ETA: 52:05:19 - loss: 1.2483 - mean_absolute_error: 1.2483debug 1.4531966183
(1, 500, 1) (1, 1)
    110/1258290 [..............................] - ETA: 52:02:51 - loss: 1.2458 - mean_absolute_error: 1.2458debug 1.4

    159/1258290 [..............................] - ETA: 51:35:36 - loss: 1.1024 - mean_absolute_error: 1.1024debug 1.4467961519
(1, 500, 1) (1, 1)
    160/1258290 [..............................] - ETA: 51:37:47 - loss: 1.0976 - mean_absolute_error: 1.0976debug 1.4467956019
(1, 500, 1) (1, 1)
    161/1258290 [..............................] - ETA: 51:37:24 - loss: 1.0917 - mean_absolute_error: 1.0917debug 1.4456995575
(1, 500, 1) (1, 1)
    162/1258290 [..............................] - ETA: 51:35:33 - loss: 1.0854 - mean_absolute_error: 1.0854debug 1.4456990075
(1, 500, 1) (1, 1)
    163/1258290 [..............................] - ETA: 51:34:10 - loss: 1.0797 - mean_absolute_error: 1.0797debug 1.4456984575
(1, 500, 1) (1, 1)
    164/1258290 [..............................] - ETA: 51:32:45 - loss: 1.0744 - mean_absolute_error: 1.0744debug 1.4456979075
(1, 500, 1) (1, 1)
    165/1258290 [..............................] - ETA: 51:31:05 - loss: 1.0703 - mean_absolute_error: 1.0703debug 1.4

    214/1258290 [..............................] - ETA: 51:29:57 - loss: 0.8723 - mean_absolute_error: 0.8723debug 1.4392974411
(1, 500, 1) (1, 1)
    215/1258290 [..............................] - ETA: 51:29:29 - loss: 0.8685 - mean_absolute_error: 0.8685debug 1.4392968911
(1, 500, 1) (1, 1)
    216/1258290 [..............................] - ETA: 51:29:04 - loss: 0.8648 - mean_absolute_error: 0.8648debug 1.4392963411
(1, 500, 1) (1, 1)
    217/1258290 [..............................] - ETA: 51:28:19 - loss: 0.8613 - mean_absolute_error: 0.8613debug 1.4392957911
(1, 500, 1) (1, 1)
    218/1258290 [..............................] - ETA: 51:27:46 - loss: 0.8574 - mean_absolute_error: 0.8574debug 1.4382997467
(1, 500, 1) (1, 1)
    219/1258290 [..............................] - ETA: 51:26:59 - loss: 0.8546 - mean_absolute_error: 0.8546debug 1.4382991967
(1, 500, 1) (1, 1)
    220/1258290 [..............................] - ETA: 51:26:21 - loss: 0.8509 - mean_absolute_error: 0.8509debug 1.4

...

   1644/1258290 [..............................] - ETA: 51:01:13 - loss: 0.1347 - mean_absolute_error: 0.1347debug 1.2531994211
(1, 500, 1) (1, 1)
   1645/1258290 [..............................] - ETA: 51:01:12 - loss: 0.1347 - mean_absolute_error: 0.1347debug 1.2531988711
(1, 500, 1) (1, 1)
   1646/1258290 [..............................] - ETA: 51:01:10 - loss: 0.1346 - mean_absolute_error: 0.1346debug 1.2531983211
(1, 500, 1) (1, 1)
   1647/1258290 [..............................] - ETA: 51:01:09 - loss: 0.1345 - mean_absolute_error: 0.1345debug 1.2531977711
(1, 500, 1) (1, 1)
   1648/1258290 [..............................] - ETA: 51:01:05 - loss: 0.1344 - mean_absolute_error: 0.1344debug 1.2531972211
(1, 500, 1) (1, 1)
   1649/1258290 [..............................] - ETA: 51:01:04 - loss: 0.1343 - mean_absolute_error: 0.1343debug 1.2531966711
(1, 500, 1) (1, 1)
   1650/1258290 [..............................] - ETA: 51:01:03 - loss: 0.1343 - mean_absolute_error: 0.1343debug 1.2

   1699/1258290 [..............................] - ETA: 50:54:18 - loss: 0.1305 - mean_absolute_error: 0.1305debug 1.2467962047
(1, 500, 1) (1, 1)
   1700/1258290 [..............................] - ETA: 50:54:16 - loss: 0.1304 - mean_absolute_error: 0.1304debug 1.2467956547
(1, 500, 1) (1, 1)
   1701/1258290 [..............................] - ETA: 50:54:12 - loss: 0.1304 - mean_absolute_error: 0.1304debug 1.2456996103
(1, 500, 1) (1, 1)
   1702/1258290 [..............................] - ETA: 50:54:11 - loss: 0.1303 - mean_absolute_error: 0.1303debug 1.2456990603
(1, 500, 1) (1, 1)
   1703/1258290 [..............................] - ETA: 50:54:07 - loss: 0.1302 - mean_absolute_error: 0.1302debug 1.2456985103
(1, 500, 1) (1, 1)
   1704/1258290 [..............................] - ETA: 50:54:03 - loss: 0.1301 - mean_absolute_error: 0.1301debug 1.2456979603
(1, 500, 1) (1, 1)
   1705/1258290 [..............................] - ETA: 50:54:02 - loss: 0.1301 - mean_absolute_error: 0.1301debug 1.2

   1754/1258290 [..............................] - ETA: 50:50:40 - loss: 0.1265 - mean_absolute_error: 0.1265debug 1.2392974939
(1, 500, 1) (1, 1)
   1755/1258290 [..............................] - ETA: 50:51:05 - loss: 0.1264 - mean_absolute_error: 0.1264debug 1.2392969439
(1, 500, 1) (1, 1)
   1756/1258290 [..............................] - ETA: 50:51:08 - loss: 0.1264 - mean_absolute_error: 0.1264debug 1.2392963939
(1, 500, 1) (1, 1)
   1757/1258290 [..............................] - ETA: 50:51:07 - loss: 0.1263 - mean_absolute_error: 0.1263debug 1.2392958439
(1, 500, 1) (1, 1)
   1758/1258290 [..............................] - ETA: 50:51:05 - loss: 0.1262 - mean_absolute_error: 0.1262debug 1.2382997995
(1, 500, 1) (1, 1)
   1759/1258290 [..............................] - ETA: 50:51:11 - loss: 0.1261 - mean_absolute_error: 0.1261debug 1.2382992495
(1, 500, 1) (1, 1)
   1760/1258290 [..............................] - ETA: 50:52:03 - loss: 0.1261 - mean_absolute_error: 0.1261debug 1.2

   1809/1258290 [..............................] - ETA: 51:03:57 - loss: 0.1228 - mean_absolute_error: 0.1228debug 1.2318987831
(1, 500, 1) (1, 1)
   1810/1258290 [..............................] - ETA: 51:03:57 - loss: 0.1227 - mean_absolute_error: 0.1227debug 1.2318982331
(1, 500, 1) (1, 1)
   1811/1258290 [..............................] - ETA: 51:03:57 - loss: 0.1227 - mean_absolute_error: 0.1227debug 1.2318976831
(1, 500, 1) (1, 1)
   1812/1258290 [..............................] - ETA: 51:04:01 - loss: 0.1226 - mean_absolute_error: 0.1226debug 1.2318971331
(1, 500, 1) (1, 1)
   1813/1258290 [..............................] - ETA: 51:04:02 - loss: 0.1225 - mean_absolute_error: 0.1225debug 1.2318965831
(1, 500, 1) (1, 1)
   1814/1258290 [..............................] - ETA: 51:04:02 - loss: 0.1225 - mean_absolute_error: 0.1225debug 1.2318960331
(1, 500, 1) (1, 1)
   1815/1258290 [..............................] - ETA: 51:04:06 - loss: 0.1224 - mean_absolute_error: 0.1224debug 1.2

KeyboardInterrupt: 

In [ ]:
next(my_training_batch_generator)